# Linear Regression ML 

RidgeCV


1. Transformer to select Detectors, Optics, Source
2. OneHot Encode All
3. RidgeCV

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV



In [17]:
def machine_learner(categorical_columns, feature):
    #read cleaned dataframe
    global df
    df = pd.read_feather('./dataframe_feather/2_detclean_dataframe.feather')
    
    #Drop R Nulls
    df = df[df['R_free'] != "NULL"]
    df = df[df['R_value'] != "NULL"]
    df = df[df['I_sigma'] != "NULL"]


    #Cast as floats
    df = df.astype({'R_free': 'float', 'R_value': 'float', 'I_sigma': 'float', 'Resolution': 'float'})
    
    ohe = OneHotEncoder()
    
    ohe.fit_transform(df[categorical_columns]) 
    
    
    global est

    est = Pipeline([
        ('categorical', OneHotEncoder()),
        ('regressor', RidgeCV())
    ])

    est.fit(df[categorical_columns], df[feature])

In [19]:
#Columns to be used in fit
categorical_columns = ['Detector', 'Optics']
feature = ['Resolution']

machine_learner(categorical_columns, feature)
res_model = est

categorical_columns = ['Detector', 'Optics']
feature = ['I_sigma']

machine_learner(categorical_columns, feature)
Isig_model = est


print("I_sigma model score: ", Isig_model.score(df[categorical_columns], df['I_sigma']))
print("Resolution model score: ", res_model.score(df[categorical_columns], df['Resolution']))


I_sigma model score:  0.0031106145987559453
Resolution model score:  0.03272348151710702


In [15]:
det_df = (df.groupby('Detector')
 .agg({'R_free': 'mean', "R_value": 'mean', "I_sigma": 'mean', "index": 'count'})
 .sort_values(["index"])
)
 
det_df.loc[det_df['index'] > 100]

,R_free,R_value,I_sigma,index
Detector,,,,
RIGAKU IMAGE PLATE,0.241515,0.194050,14.548218,101
RIGAKU CCD,0.237735,0.196098,13.483039,102
BRUKER DIP-6040,0.235881,0.194752,18.831231,109
REMARK 200 DETECTOR MANUFACTURER : ADSC QUANTUM 315R,0.239895,0.200868,18.282228,114
CS-PAD CXI-1,0.250575,0.214125,8.806217,120
...,...,...,...,...
MARRESEARCH,0.233136,0.192455,22.703894,4976
MARMOSAIC 225 MM CCD,0.224619,0.187112,16.491488,5177
RAYONIX MX-300,0.225675,0.188991,17.039416,5676


In [17]:
df['Detector'].unique()

array(['DECTRIS PILATUS 6M', 'DECTRIS EIGER X 4M', 'DECTRIS EIGER X 16M',
       'DECTRIS PILATUS 6M-F', 'DECTRIS EIGER2 XE 16M',
       'DECTRIS PILATUS3 6M', 'MAR CCD 130 MM', 'DECTRIS EIGER X 9M',
       'DECTRIS PILATUS 300K', 'DECTRIS EIGER2 X 9M',
       'DECTRIS PILATUS3 2M', 'ADSC QUANTUM 210R', 'RAYONIX MX300-HS',
       'RIGAKU SATURN 944+', 'DECTRIS PILATUS 2M', 'ADSC QUANTUM 270',
       'DECTRIS PILATUS3 X 6M', 'DECTRIS PILATUS3 S 6M',
       'ADSC QUANTUM 315R', 'MARMOSAIC 225 MM CCD', 'MAR CCD 165 MM',
       'RIGAKU RAXIS IV++', 'RAYONIX MX-300', 'RDI CMOS_8M',
       'BRUKER PLATINUM 135', 'DECTRIS EIGER2 X 16M',
       'DECTRIS PILATUS 200K', 'RIGAKU RAXIS IV', 'RIGAKU',
       'AGILENT ATLAS CCD', 'MAR SCANNER 300 MM PLATE', 'MARRESEARCH',
       'ADSC QUANTUM 4R', 'RIGAKU RAXIS II', 'ADSC', 'ADSC CCD',
       'CUSTOM-MADE',
       'REMARK 200  DETECTOR MANUFACTURER          : ADSC QUANTUM 315',
       'RIGAKU RAXIS IIC',
       'REMARK 200  DETECTOR MANUFACTURER    

In [6]:

df['Optics'].unique()

array(['NULL', 'RH COATED COLLIMATING MIRRORS, K',
       'FLAT SIDE-DEFLECTING, RH-COATED', ..., 'CCD ARRAY',
       'BIMORPH MIRRORS', 'TORODIAL FOCUSING MIRRORS'], dtype=object)

In [33]:
#Test your setups
# 'DECTRIS EIGER X 16M'
# 'DECTRIS PILATUS 6M'

detector = 'DECTRIS EIGER X 9M'
mirror = 'BIMORPH MIRRORS'

print("resolution estimate: ", res_model.predict([[detector, mirror]])[0])
print("I/sigma estimate: ", Isig_model.predict([[detector, mirror]])[0])

resolution estimate:  [2.07249358]
I/sigma estimate:  [19.28237408]
